# Utils

> Some helper methods and so on ... everything really

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#|default_exp utils
#|export
from pathlib import Path
import logging
import sys

def get_logger(name   = None, 
               form   = '[%(name)s] %(message)s', 
               level  = logging.INFO, 
               stream = None):
    
    if name is None: name = __name__
    logger = logging.getLogger(name)
    logger.setLevel(level)

    handler = logging.StreamHandler(stream)    
    formatter = logging.Formatter(form)
    handler.setFormatter(formatter)
    
    logger.handlers.clear()
    logger.addHandler(handler)
    
    return logger

In [ ]:
logger = get_logger("NBX.utils")
log = logger.info
log("test")

[NBX.utils] test


In [ ]:
#|export
class Bunch(dict):
    def __init__(self, **kwargs):
        super().__init__(kwargs)

    def __setattr__(self, key, value):
        self[key] = value

    def __dir__(self):
        return self.keys()

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(key)

In [ ]:
#|export
import nbx

def nbx_home():
    return Path(nbx.__file__).parents[1]

def nbx_lib():
    return Path(nbx.__file__).parents[0]

In [ ]:
nbx_home(), nbx_lib()

(PosixPath('/Users/mirko/Workspace/nbx'),
 PosixPath('/Users/mirko/Workspace/nbx/nbx'))

In [ ]:
#|export
import time
import datetime

def timestamp(form='%Y-%m-%d_%H:%M:%S'):
    t = time.time()
    return datetime.datetime.fromtimestamp(t).strftime(form)

In [ ]:
#|export
import numpy as np
from functools import reduce, partial

def listmap(f, arr):
    return list(map(f,arr))

def arrmap(f,arr):
    return np.array(listmap(f,arr))

def is_list(val):
    return hasattr(val, '__iter__') and type(val) != str

In [ ]:
#|export
def bunch_of_lists(list_of_dicts, keys=None):
    y = {}    
    if keys is None: keys = list_of_dicts[0].keys()
    for k in keys: y[k] = arrmap(get(k), list_of_dicts)
    return Bunch(**y)

In [ ]:
#|export
def sliding_window_ind(T, n, step, remainder=False):    
    if n > T:
        if remainder: return [], np.arange(T)
        else: return []
    
    I   = np.tile(np.arange(n)[:,None], T-n+1) + np.arange(T-n+1)[None]
    I   = I.T
    sub = np.arange(len(I), step=step)
    I   = I[sub]
    
    if remainder:
        i = I[-1]
        return I, np.arange(i[0] + step, T)
    else:
        return I

In [ ]:
w = sliding_window_ind(T=10, n=3, step=2)
print(w.shape)
w

(4, 3)


array([[0, 1, 2],
       [2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

In [ ]:
#|export
import subprocess

def run_bash(command):
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE)
    result = result.stdout.decode('UTF-8')
    return result.rstrip().split("\n")

In [ ]:
run_bash("ls")

['01_utils.ipynb',
 '02_fileio.ipynb',
 '03_templ.ipynb',
 '04_confspace.ipynb',
 '05_nbparser.ipynb',
 '06_my_parser.ipynb',
 '_01_logging.ipynb',
 '_04_nbparser - original.ipynb',
 '_04b_nbparser.ipynb',
 '_06_my_parser-Copy1.ipynb',
 'index.ipynb',
 'nbx_example']

In [ ]:
#|export
import ipynbname

def this_nb_to_html(name="{nb}", pre="_", suff=""):
    """
    If called from within a notebook converts this 
    notebook to html and returns the html file name. 
    
    The html file name can be formated using 
    the notebook name `nb` and a current time stamp `t`.
    """
    t      = timestamp()
    path   = ipynbname.path().parent
    name   = ipynbname.name() # nb name
    rename = (pre + name + suff).format(nb=name, t=t) # renamed
    
    a = path/f"{name}.ipynb"
    b = path/f"{name}.html"
    c = path/f"{rename}.html"

    log(f"...Converting `./{a.name}` to `./{c.name}`.")
    run_bash(f"jupyter nbconvert {a} --to html ")
    run_bash(f"mv {b} {c}")
    
    return c

In [ ]:
nb = this_nb_to_html(suff="_[{t}]")
!open $nb
!rm $nb

[NBX.utils] ...Converting `./01_utils.ipynb` to `./_01_utils_[2023-02-06_14:55:06].html`.
[NbConvertApp] Converting notebook /Users/mirko/Workspace/nbx/notebooks/01_utils.ipynb to html
[NbConvertApp] Writing 614365 bytes to /Users/mirko/Workspace/nbx/notebooks/01_utils.html


In [ ]:
#|export
import inspect

def defaultfrom(c):
    """
    Binds values from context to 
    default KEYWORD-ONLY (!) arguments.
    """
    def deco(f):
        kw = inspect.getfullargspec(f).kwonlyargs
        kw_in_c = []
        for k in kw:
            if k in c: kw_in_c.append(k)


        def g(*args, **kwargs):
            for k in kw_in_c: 
                if k not in kwargs: kwargs[k] = c[k]

            return f(*args, **kwargs)

        
        return g
    
    return deco

In [ ]:
#|export
def default_args(f):
    sig = inspect.signature(f)
    d = {}
    
    for k,v in sig.parameters.items():
        if v.default is not inspect.Parameter.empty:
            d[k] = v.default
            
    return d



In [ ]:
def f(x, y=1, *, z="two", w=None):  return x
default_args(f)

{'y': 1, 'z': 'two', 'w': None}

In [ ]:
#|export
import ipynbname
import json
def load_nb(fname):
    """Loads a a ipynotebook-bunch""" 
    try:
        nbdict = json.loads(fname)
    except:
        nbdict = json.load(open(fname,'r',encoding="utf-8"))
        
    nbdict["fname"] = str(fname)
    return Bunch(**nbdict)

In [ ]:
nb = load_nb("nbx_example/test_notebook.ipynb")
nb.cells[0]

{'cell_type': 'markdown',
 'id': '89c3991f',
 'metadata': {},
 'source': ['# Test Notebook for `nbx`']}

In [ ]:
import ipynbname
ipynbname.path(), ipynbname.name()

(PosixPath('/Users/mirko/Workspace/nbx/notebooks/01_utils.ipynb'), '01_utils')